# TODO:
### Fixa weather API
### Fixa dummy temp-sensor
### Fixa API som tar emot location och väderdata och temperatur
### Fixa att API uppdaterar inomhustemperatur

### Geolocation API

# API

In [17]:
# API boiler plate.
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, UUID4

from uuid import uuid4

app = FastAPI()

class command(BaseModel):
    q_id: UUID4
    request: str
    answer: str | None

class createCommand(BaseModel):
    request: str

#TODO: Define classes!

In [21]:
# API functions.
@app.get("/")
def read_root():
    return {"message": "Head message."}


import requests
import os
from dotenv import load_dotenv

# User enters location. Function returns current outdoor temp.
@app.post("/location")
def enter_location(house_location: str):
    # Load environmental variables
    load_dotenv()

    # Get the API key for geocoding
    geocode_apikey = os.getenv('GEOCODE_API')
    if not geocode_apikey:
        return {"error": "Geocode API key not found"}

    # Form the geocoding URL to fetch latitude and longitude
    geocode_url = f"https://geocode.maps.co/search?q={house_location}&api_key={geocode_apikey}"
    
    try:
        geocode_response = requests.get(geocode_url)
        geocode_response.raise_for_status()  # Raises HTTPError for bad requests (4XX or 5XX)
        geocode_data = geocode_response.json()
        lat = geocode_data[0]['lat']
        lon = geocode_data[0]['lon']
    except requests.RequestException as e:
        return {"error": str(e)}
    except (IndexError, KeyError):
        return {"error": "Failed to get valid location data"}

    # Form the URL to fetch the current temperature
    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m&hourly=temperature_2m"
    
    try:
        weather_response = requests.get(weather_url)
        weather_response.raise_for_status()
        weather_data = weather_response.json()
        temp = weather_data['hourly']['temperature_2m']
    except requests.RequestException as e:
        return {"error": str(e)}
    except KeyError:
        return {"error": "Failed to get temperature data"}

    return {"temperature": temp}